In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc numpy pandas

# אלגוריתם שור

*הערכת שימוש: שלוש שניות על מעבד Eagle r3 (הערה: זוהי הערכה בלבד. זמן הריצה שלך עשוי להשתנות.)*

[אלגוריתם שור,](https://epubs.siam.org/doi/abs/10.1137/S0036144598347011) שפותח על ידי פיטר שור בשנת 1994, הוא אלגוריתם קוונטי פורץ דרך לפירוק גורמים של מספרים שלמים בזמן פולינומי. משמעותו טמונה ביכולתו לפרק גורמים של מספרים שלמים גדולים במהירות אקספוננציאלית לעומת כל אלגוריתם קלאסי ידוע, תוך איום על אבטחת מערכות קריפטוגרפיות בשימוש נרחב כמו RSA, המסתמכות על הקושי של פירוק גורמים של מספרים גדולים. באמצעות פתרון יעיל של בעיה זו על מחשב קוונטי חזק מספיק, אלגוריתם שור עשוי לחולל מהפכה בתחומים כגון קריפטוגרפיה, אבטחת סייבר ומתמטיקה חישובית, תוך הדגשת הכוח הטרנספורמטיבי של חישוב קוונטי.

מדריך זה מתמקד בהדגמת אלגוריתם שור באמצעות פירוק הגורמים של 15 במחשב קוונטי.

ראשית, אנו מגדירים את בעיית מציאת הסדר ובונים מעגלים תואמים מפרוטוקול הערכת הפאזה הקוונטי. לאחר מכן, אנו מריצים את מעגלי מציאת הסדר על חומרה אמיתית תוך שימוש במעגלים בעומק הקצר ביותר שאנו יכולים לתרגם. החלק האחרון משלים את אלגוריתם שור על ידי חיבור בעיית מציאת הסדר לפירוק גורמים של מספרים שלמים.

אנו מסיימים את המדריך בדיון על הדגמות אחרות של אלגוריתם שור על חומרה אמיתית, תוך התמקדות הן ביישומים גנריים והן באלו המותאמים לפירוק גורמים של מספרים שלמים ספציפיים כגון 15 ו-21.
הערה: מדריך זה מתמקד יותר ביישום ובהדגמה של המעגלים הנוגעים לאלגוריתם שור. למשאב חינוכי מעמיק על החומר, אנא עיין בקורס [יסודות האלגוריתמים הקוונטיים](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction) מאת ד"ר ג'ון ווטרוס, ומאמרים בסעיף [הפניות](#references).
### דרישות
לפני תחילת מדריך זה, ודא שיש לך את הדברים הבאים מותקנים:
- Qiskit SDK v2.0 ומעלה, עם תמיכה ב-[ויזואליזציה](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.40 ומעלה (`pip install qiskit-ibm-runtime`)
### הכנה

In [ ]:
import numpy as np
import pandas as pd
from fractions import Fraction
from math import floor, gcd, log

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT, UnitaryGate
from qiskit.transpiler import CouplingMap, generate_preset_pass_manager
from qiskit.visualization import plot_histogram

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

## שלב 1: מיפוי קלט קלאסי לבעיה קוונטית
### רקע
אלגוריתם שור לפירוק גורמים של מספרים שלמים משתמש בבעיה מתווכת המכונה בעיית *מציאת הסדר*. בחלק זה, אנו מדגימים כיצד לפתור את בעיית מציאת הסדר באמצעות *הערכת פאזה קוונטית*.
### בעיית הערכת פאזה
בבעיית הערכת הפאזה, ניתן לנו מצב קוונטי $\ket{\psi}$ של $n$ קיוביטים, יחד עם מעגל קוונטי יוניטרי הפועל על $n$ קיוביטים. ניתנת לנו ההבטחה ש-$\ket{\psi}$ הוא וקטור עצמי של המטריצה היוניטרית $U$ המתארת את פעולת המעגל, והמטרה שלנו היא לחשב או לקרב את ערך העצמי $\lambda = e^{2 \pi i \theta}$ שאליו $\ket{\psi}$ מתאים. במילים אחרות, המעגל צריך להוציא קירוב למספר $\theta \in [0, 1)$ המקיים $$U \ket{\psi}= e^{2 \pi i \theta} \ket{\psi}.$$
מטרת מעגל הערכת הפאזה היא לקרב את $\theta$ ב-$m$ ביטים. מבחינה מתמטית, אנו רוצים למצוא $y$ כך ש-$\theta \approx y / 2^m$, כאשר $y \in {0, 1, 2, \dots, 2^{m-1}}$. התמונה הבאה מציגה את המעגל הקוונטי שמעריך את $y$ ב-$m$ ביטים על ידי ביצוע מדידה על $m$ קיוביטים.
![מעגל הערכת פאזה קוונטי](../learning/images/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/phase-estimation-procedure.svg)
במעגל שלמעלה, $m$ הקיוביטים העליונים מאותחלים במצב $\ket{0^m}$, ו-$n$ הקיוביטים התחתונים מאותחלים ב-$\ket{\psi}$, שמובטח שהוא וקטור עצמי של $U$. המרכיב הראשון במעגל הערכת הפאזה הם הפעולות היוניטריות המבוקרות האחראיות לביצוע *בעיטת פאזה* לקיוביט הבקרה המתאים שלהן. יוניטריים מבוקרים אלו מועלים בחזקה בהתאם למיקום של קיוביט הבקרה, החל מהביט המשמעותי ביותר ועד לביט הפחות משמעותי. מכיוון ש-$\ket{\psi}$ הוא וקטור עצמי של $U$, המצב של $n$ הקיוביטים התחתונים אינו מושפע מפעולה זו, אך מידע הפאזה של ערך העצמי מתפשט ל-$m$ הקיוביטים העליונים.
מסתבר שלאחר פעולת בעיטת הפאזה באמצעות יוניטריים מבוקרים, כל המצבים האפשריים של $m$ הקיוביטים העליונים אורתונורמליים זה לזה עבור כל וקטור עצמי $\ket{\psi}$ של היוניטרי $U$. לכן, מצבים אלו ניתנים להבחנה באופן מושלם, ואנו יכולים לסובב את הבסיס שהם יוצרים בחזרה לבסיס החישובי כדי לבצע מדידה. ניתוח מתמטי מראה שמטריצת סיבוב זו מתאימה לטרנספורמציה של פורייה הקוונטית (QFT) ההופכית במרחב הילברט בעל מימד $2^m$. האינטואיציה מאחורי זה היא שהמבנה המחזורי של אופרטורי ההעלאה בחזקה המודולרית מקודד במצב הקוונטי, וה-QFT ממיר מחזוריות זו לפסגות ניתנות למדידה בתחום התדר.

להבנה מעמיקה יותר של מדוע נעשה שימוש במעגל QFT באלגוריתם שור, אנו מפנים את הקורא לקורס [יסודות האלגוריתמים הקוונטיים](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction).
כעת אנו מוכנים להשתמש במעגל הערכת הפאזה למציאת סדר.
### בעיית מציאת הסדר
כדי להגדיר את בעיית מציאת הסדר, אנו מתחילים עם כמה מושגי תורת המספרים. ראשית, עבור כל מספר שלם חיובי נתון $N$, נגדיר את הקבוצה $\mathbb{Z}_N$ כ-$$\mathbb{Z}_N = {0, 1, 2, \dots, N-1}.$$
כל הפעולות האריתמטיות ב-$\mathbb{Z}_N$ מבוצעות מודולו $N$. בפרט, כל האלמנטים $a \in \mathbb{Z}_n$ שהם זרים זה לזה עם $N$ הם מיוחדים ומהווים את $\mathbb{Z}^*_N$ כ-$$\mathbb{Z}^*_N = { a \in \mathbb{Z}_N : \mathrm{gcd}(a, N)=1 }.$$
עבור אלמנט $a \in \mathbb{Z}^*_N$, המספר השלם החיובי הקטן ביותר $r$ כך ש-$$a^r \equiv 1 \; (\mathrm{mod} \; N)$$ מוגדר כ-*סדר* של $a$ מודולו $N$. כפי שנראה מאוחר יותר, מציאת הסדר של $a \in \mathbb{Z}^*_N$ תאפשר לנו לפרק גורמים של $N$.
כדי לבנות את מעגל מציאת הסדר ממעגל הערכת הפאזה, אנו זקוקים לשני שיקולים. ראשית, אנו צריכים להגדיר את היוניטרי $U$ שיאפשר לנו למצוא את הסדר $r$, ושנית, אנו צריכים להגדיר וקטור עצמי $\ket{\psi}$ של $U$ כדי להכין את המצב ההתחלתי של מעגל הערכת הפאזה.

כדי לחבר את בעיית מציאת הסדר להערכת פאזה, אנו שוקלים את הפעולה המוגדרת על מערכת שמצביה הקלאסיים מתאימים ל-$\mathbb{Z}_N$, שבה אנו מכפילים באלמנט קבוע $a \in \mathbb{Z}^*_N$. בפרט, אנו מגדירים את אופרטור הכפל $M_a$ כך ש-$$M_a \ket{x} = \ket{ax \; (\mathrm{mod} \; N)}$$ עבור כל $x \in \mathbb{Z}_N$. שימו לב שזה משתמע שאנו לוקחים את המכפלה מודולו $N$ בתוך ה-ket בצד ימין של המשוואה. ניתוח מתמטי מראה ש-$M_a$ הוא אופרטור יוניטרי. יתרה מכך, מסתבר ש-$M_a$ יש זוגות וקטור עצמי וערך עצמי המאפשרים לנו לחבר את הסדר $r$ של $a$ לבעיית הערכת הפאזה. באופן ספציפי, עבור כל בחירה של $j \in {0, \dots, r-1}$, יש לנו $$\ket{\psi_j} = \frac{1}{\sqrt{r}} \sum^{r-1}_{k=0} \omega^{-jk}_{r} \ket{a^k}$$ הוא וקטור עצמי של $M_a$ שערך העצמי המתאים לו הוא $\omega^{j}_{r}$, כאשר $$\omega^{j}_{r} = e^{2 \pi i \frac{j}{r}}.$$
על ידי תצפית, אנו רואים שזוג וקטור עצמי/ערך עצמי נוח הוא המצב $\ket{\psi_1}$ עם $\omega^{1}_{r} = e^{2 \pi i \frac{1}{r}}$. לכן, אם היינו יכולים למצוא את הוקטור העצמי $\ket{\psi_1}$, היינו יכולים להעריך את הפאזה $\theta=1/r$ עם המעגל הקוונטי שלנו ולכן לקבל הערכה של הסדר $r$. עם זאת, זה לא קל לעשות זאת, ואנחנו צריכים לשקול אלטרנטיבה.

בואו נבחן מה יהיה תוצאת המעגל אם נכין את המצב החישובי $\ket{1}$ כמצב ההתחלתי. זה לא מצב עצמי של $M_a$, אבל הוא הסופרפוזיציה האחידה של מצבים עצמיים שתיארנו לעיל. במילים אחרות, היחס הבא מתקיים. $$ \ket{1} = \frac{1}{\sqrt{r}} \sum^{r-1}_{k=0} \ket{\psi_k} $$
המשמעות של המשוואה לעיל היא שאם נקבע את המצב ההתחלתי ל-$\ket{1}$, נקבל בדיוק את אותה תוצאת מדידה כאילו בחרנו $k \in { 0, \dots, r-1}$ באופן אחיד אקראי והשתמשנו ב-$\ket{\psi_k}$ כוקטור עצמי במעגל הערכת הפאזה. במילים אחרות, מדידה של $m$ הקיוביטים העליונים מניבה קירוב $y / 2^m$ לערך $k / r$ כאשר $k \in { 0, \dots, r-1}$ נבחר באופן אחיד אקראי. זה מאפשר לנו ללמוד את $r$ עם רמת ביטחון גבוהה לאחר מספר הרצות עצמאיות, שזו הייתה המטרה שלנו.
### אופרטורי העלאה בחזקה מודולריים
עד כה, קישרנו את בעיית הערכת הפאזה לבעיית מציאת הסדר על ידי הגדרת $U = M_a$ ו-$\ket{\psi} = \ket{1}$ במעגל הקוונטי שלנו. לכן, המרכיב האחרון שנותר הוא למצוא דרך יעילה להגדיר חזקות מודולריות של $M_a$ כ-$M_a^k$ עבור $k = 1, 2, 4, \dots, 2^{m-1}$.
כדי לבצע חישוב זה, אנו מוצאים שעבור כל חזקה $k$ שאנו בוחרים, אנו יכולים ליצור מעגל עבור $M_a^k$ לא על ידי איטרציה $k$ פעמים של המעגל עבור $M_a$, אלא על ידי חישוב $b = a^k \; \mathrm{mod} \; N$ ולאחר מכן שימוש במעגל עבור $M_b$. מכיוון שאנו זקוקים רק לחזקות שהן בעצמן חזקות של 2, אנו יכולים לעשות זאת באופן יעיל קלאסית באמצעות העלאה בריבוע איטרטיבית.
## שלב 2: אופטימיזציה של הבעיה לביצוע על חומרה קוונטית
### דוגמה ספציפית עם $N = 15$ ו-$a=2$
אנו יכולים לעצור כאן כדי לדון בדוגמה ספציפית ולבנות את מעגל מציאת הסדר עבור $N=15$. שימו לב ש-$a \in \mathbb{Z}_N^*$ לא-טריוויאליים אפשריים עבור $N=15$ הם $a \in {2, 4, 7, 8, 11, 13, 14 }$. בדוגמה זו, אנו בוחרים $a=2$. אנו נבנה את האופרטור $M_2$ ואת אופרטורי ההעלאה בחזקה המודולריים $M_2^k$.
פעולת $M_2$ על מצבי הבסיס החישובי היא כדלקמן.
$$M_2 \ket{0} = \ket{0} \quad M_2 \ket{5} = \ket{10} \quad M_2 \ket{10} = \ket{5}$$
$$M_2 \ket{1} = \ket{2} \quad M_2 \ket{6} = \ket{12} \quad M_2 \ket{11} = \ket{7}$$
$$M_2 \ket{2} = \ket{4} \quad M_2 \ket{7} = \ket{14} \quad M_2 \ket{12} = \ket{9}$$
$$M_2 \ket{3} = \ket{6} \quad M_2 \ket{8} = \ket{1} \quad M_2 \ket{13} = \ket{11}$$
$$M_2 \ket{4} = \ket{8} \quad M_2 \ket{9} = \ket{3} \quad M_2 \ket{14} = \ket{13}$$
על ידי תצפית, אנו יכולים לראות שמצבי הבסיס מעורבבים, כך שיש לנו מטריצת תמורה. אנו יכולים לבנות פעולה זו על ארבעה קיוביטים עם שערי swap. להלן, אנו בונים את $M_2$ ואת הפעולות controlled-$M_2$.

In [2]:
def M2mod15():
    """
    M2 (mod 15)
    """
    b = 2
    U = QuantumCircuit(4)

    U.swap(2, 3)
    U.swap(1, 2)
    U.swap(0, 1)

    U = U.to_gate()
    U.name = f"M_{b}"

    return U

In [3]:
# Get the M2 operator
M2 = M2mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(4)
circ.compose(M2, inplace=True)
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0a8885f1-91d4-40bd-912d-dc5eea05f5bd-0.avif" alt="Output of the previous code cell" />

In [4]:
def controlled_M2mod15():
    """
    Controlled M2 (mod 15)
    """
    b = 2
    U = QuantumCircuit(4)

    U.swap(2, 3)
    U.swap(1, 2)
    U.swap(0, 1)

    U = U.to_gate()
    U.name = f"M_{b}"
    c_U = U.control()

    return c_U

In [5]:
# Get the controlled-M2 operator
controlled_M2 = controlled_M2mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(5)
circ.compose(controlled_M2, inplace=True)
circ.decompose(reps=1).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/ab7fe331-2f9e-47ca-ba3b-f5d67992062a-0.avif" alt="Output of the previous code cell" />

![פלט של תא הקוד הקודם](../docs/images/tutorials/shors-algorithm/extracted-outputs/ab7fe331-2f9e-47ca-ba3b-f5d67992062a-0.avif)

שערים הפועלים על יותר משני קיוביטים יפורקו עוד יותר לשערים של שני קיוביטים.

In [6]:
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/13b4841d-a4ac-46bd-b4d0-d111b3017189-0.avif" alt="Output of the previous code cell" />

![פלט של תא הקוד הקודם](../docs/images/tutorials/shors-algorithm/extracted-outputs/13b4841d-a4ac-46bd-b4d0-d111b3017189-0.avif)

כעת אנו צריכים לבנות את אופרטורי ההעלאה בחזקה המודולריים. כדי לקבל דיוק מספיק בהערכת הפאזה, נשתמש בשמונה קיוביטים למדידת ההערכה. לכן, אנו צריכים לבנות $M_b$ עם $b = a^{2^k} \; (\mathrm{mod} \; N)$ עבור כל $k = 0, 1, \dots, 7$.

In [7]:
def a2kmodN(a, k, N):
    """Compute a^{2^k} (mod N) by repeated squaring"""
    for _ in range(k):
        a = int(np.mod(a**2, N))
    return a

In [8]:
k_list = range(8)
b_list = [a2kmodN(2, k, 15) for k in k_list]

print(b_list)

[2, 4, 1, 1, 1, 1, 1, 1]


As we can see from the list of $b$ values, in addition to $M_2$ that we previously constructed, we also need to build $M_4$ and $M_1$. Note that $M_1$ acts trivially on the computational basis states, so it is simply the identity operator.

$M_4$ acts on the computational basis states as follows.
$$M_4 \ket{0} = \ket{0} \quad M_4 \ket{5} = \ket{5} \quad M_4 \ket{10} = \ket{10}$$
$$M_4 \ket{1} = \ket{4} \quad M_4 \ket{6} = \ket{9} \quad M_4 \ket{11} = \ket{14}$$
$$M_4 \ket{2} = \ket{8} \quad M_4 \ket{7} = \ket{13} \quad M_4 \ket{12} = \ket{3}$$
$$M_4 \ket{3} = \ket{12} \quad M_4 \ket{8} = \ket{2} \quad M_4 \ket{13} = \ket{7}$$
$$M_4 \ket{4} = \ket{1} \quad M_4 \ket{9} = \ket{6} \quad M_4 \ket{14} = \ket{11}$$

Therefore, this permutation can be constructed with the following swap operation.

In [9]:
def M4mod15():
    """
    M4 (mod 15)
    """
    b = 4
    U = QuantumCircuit(4)

    U.swap(1, 3)
    U.swap(0, 2)

    U = U.to_gate()
    U.name = f"M_{b}"

    return U

In [10]:
# Get the M4 operator
M4 = M4mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(4)
circ.compose(M4, inplace=True)
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/be041e3d-28b1-453e-983e-184c2366aeb9-0.avif" alt="Output of the previous code cell" />

In [11]:
def controlled_M4mod15():
    """
    Controlled M4 (mod 15)
    """
    b = 4
    U = QuantumCircuit(4)

    U.swap(1, 3)
    U.swap(0, 2)

    U = U.to_gate()
    U.name = f"M_{b}"
    c_U = U.control()

    return c_U

In [12]:
# Get the controlled-M4 operator
controlled_M4 = controlled_M4mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(5)
circ.compose(controlled_M4, inplace=True)
circ.decompose(reps=1).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/8d943b00-a502-4157-8a0d-13fb1f55e705-0.avif" alt="Output of the previous code cell" />

Gates acting on more than two qubits will be further decomposed into two-qubit gates.

In [13]:
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/68399eef-5e55-4c95-a8a4-c8efaebd34b9-0.avif" alt="Output of the previous code cell" />

![פלט של תא הקוד הקודם](../docs/images/tutorials/shors-algorithm/extracted-outputs/8d943b00-a502-4157-8a0d-13fb1f55e705-0.avif)

שערים הפועלים על יותר משני קיוביטים יפורקו עוד יותר לשערים של שני קיוביטים.

In [14]:
def mod_mult_gate(b, N):
    """
    Modular multiplication gate from permutation matrix.
    """
    if gcd(b, N) > 1:
        print(f"Error: gcd({b},{N}) > 1")
    else:
        n = floor(log(N - 1, 2)) + 1
        U = np.full((2**n, 2**n), 0)
        for x in range(N):
            U[b * x % N][x] = 1
        for x in range(N, 2**n):
            U[x][x] = 1
        G = UnitaryGate(U)
        G.name = f"M_{b}"
        return G

In [15]:
# Let's build M2 using the permutation matrix definition
M2_other = mod_mult_gate(2, 15)

# Add it to a circuit
circ = QuantumCircuit(4)
circ.compose(M2_other, inplace=True)
circ = circ.decompose()

# Transpile the circuit and get the depth
coupling_map = CouplingMap.from_line(4)
pm = generate_preset_pass_manager(coupling_map=coupling_map)
transpiled_circ = pm.run(circ)

print(f"qubits: {circ.num_qubits}")
print(
    f"2q-depth: {transpiled_circ.depth(lambda x: x.operation.num_qubits==2)}"
)
print(f"2q-size: {transpiled_circ.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_circ.count_ops()}")
transpiled_circ.decompose().draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

qubits: 4
2q-depth: 94
2q-size: 96
Operator counts: OrderedDict({'cx': 45, 'swap': 32, 'u': 24, 'u1': 7, 'u3': 4, 'unitary': 3, 'circuit-335': 1, 'circuit-338': 1, 'circuit-341': 1, 'circuit-344': 1, 'circuit-347': 1, 'circuit-350': 1, 'circuit-353': 1, 'circuit-356': 1, 'circuit-359': 1, 'circuit-362': 1, 'circuit-365': 1, 'circuit-368': 1, 'circuit-371': 1, 'circuit-374': 1, 'circuit-377': 1, 'circuit-380': 1})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/c184f6dd-9f80-4487-ac0b-0dd94170b0f0-1.avif" alt="Output of the previous code cell" />

Let's compare these counts with the compiled circuit depth of our manual implementation of the $M_2$ gate.

In [16]:
# Get the M2 operator from our manual construction
M2 = M2mod15()

# Add it to a circuit
circ = QuantumCircuit(4)
circ.compose(M2, inplace=True)
circ = circ.decompose(reps=3)

# Transpile the circuit and get the depth
coupling_map = CouplingMap.from_line(4)
pm = generate_preset_pass_manager(coupling_map=coupling_map)
transpiled_circ = pm.run(circ)

print(f"qubits: {circ.num_qubits}")
print(
    f"2q-depth: {transpiled_circ.depth(lambda x: x.operation.num_qubits==2)}"
)
print(f"2q-size: {transpiled_circ.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_circ.count_ops()}")
transpiled_circ.draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

qubits: 4
2q-depth: 9
2q-size: 9
Operator counts: OrderedDict({'cx': 9})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0235c931-0adb-4972-9fce-32a0341822bf-1.avif" alt="Output of the previous code cell" />

As we can see, the permutation matrix approach resulted in a significantly deep circuit even for a single $M_2$ gate compared to our manual implementation of it. Therefore, we will continue with our previous implementation of the $M_b$ operations.

Now, we are ready to construct the full order finding circuit using our previously defined controlled modular exponentiation operators. In the following code, we also import the [QFT circuit](/docs/api/qiskit/qiskit.circuit.library.QFT) from the Qiskit Circuit library, which uses Hadamard gates on each qubit, a series of controlled-U1 (or Z, depending on the phase) gates, and a layer of swap gates.

In [17]:
# Order finding problem for N = 15 with a = 2
N = 15
a = 2

# Number of qubits
num_target = floor(log(N - 1, 2)) + 1  # for modular exponentiation operators
num_control = 2 * num_target  # for enough precision of estimation

# List of M_b operators in order
k_list = range(num_control)
b_list = [a2kmodN(2, k, 15) for k in k_list]

# Initialize the circuit
control = QuantumRegister(num_control, name="C")
target = QuantumRegister(num_target, name="T")
output = ClassicalRegister(num_control, name="out")
circuit = QuantumCircuit(control, target, output)

# Initialize the target register to the state |1>
circuit.x(num_control)

# Add the Hadamard gates and controlled versions of the
# multiplication gates
for k, qubit in enumerate(control):
    circuit.h(k)
    b = b_list[k]
    if b == 2:
        circuit.compose(
            M2mod15().control(), qubits=[qubit] + list(target), inplace=True
        )
    elif b == 4:
        circuit.compose(
            M4mod15().control(), qubits=[qubit] + list(target), inplace=True
        )
    else:
        continue  # M1 is the identity operator

# Apply the inverse QFT to the control register
circuit.compose(QFT(num_control, inverse=True), qubits=control, inplace=True)

# Measure the control register
circuit.measure(control, output)

circuit.draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0e854aed-c11b-494c-8c80-adeb8eb0e8fe-0.avif" alt="Output of the previous code cell" />

![פלט של תא הקוד הקודם](../docs/images/tutorials/shors-algorithm/extracted-outputs/c184f6dd-9f80-4487-ac0b-0dd94170b0f0-1.avif)

בואו נשווה את הספירות האלו עם עומק המעגל המקומפל של היישום הידני שלנו של שער $M_2$.

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_marrakesh")
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)

transpiled_circuit = pm.run(circuit)

print(
    f"2q-depth: {transpiled_circuit.depth(lambda x: x.operation.num_qubits==2)}"
)
print(
    f"2q-size: {transpiled_circuit.size(lambda x: x.operation.num_qubits==2)}"
)
print(f"Operator counts: {transpiled_circuit.count_ops()}")
transpiled_circuit.draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

2q-depth: 187
2q-size: 260
Operator counts: OrderedDict({'sx': 521, 'rz': 354, 'cz': 260, 'measure': 8, 'x': 4})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/95925dd5-7ba9-4746-b96e-ba50400fa5ac-1.avif" alt="Output of the previous code cell" />

## Step 3: Execute using Qiskit primitives

First, we discuss what we would theoretically obtain if we ran this circuit on an ideal simulator. Below, we have a set of simulation results of the above circuit using 1024 shots. As we can see, we get an approximately uniform distribution over four bitstrings over the control qubits.

In [19]:
# Obtained from the simulator
counts = {"00000000": 264, "01000000": 268, "10000000": 249, "11000000": 243}

In [20]:
plot_histogram(counts)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0d6d2702-02e4-47de-8f7e-0b256657ef0f-0.avif" alt="Output of the previous code cell" />

![פלט של תא הקוד הקודם](../docs/images/tutorials/shors-algorithm/extracted-outputs/0e854aed-c11b-494c-8c80-adeb8eb0e8fe-0.avif)

שימו לב שהשמטנו את פעולות ההעלאה בחזקה המודולריות המבוקרות משאר קיוביטי הבקרה מכיוון ש-$M_1$ הוא אופרטור הזהות.
שימו לב שמאוחר יותר במדריך זה, נריץ את המעגל הזה על ה-backend `ibm_marrakish`. כדי לעשות זאת, אנו מתרגמים את המעגל בהתאם ל-backend ספציפי זה ומדווחים על עומק המעגל ומספר השערים.

In [21]:
# Rows to be displayed in table
rows = []
# Corresponding phase of each bitstring
measured_phases = []

for output in counts:
    decimal = int(output, 2)  # Convert bitstring to decimal
    phase = decimal / (2**num_control)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append(
        [
            f"{output}(bin) = {decimal:>3}(dec)",
            f"{decimal}/{2 ** num_control} = {phase:.2f}",
        ]
    )

# Print the rows in a table
headers = ["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

            Register Output           Phase
0  00000000(bin) =   0(dec)    0/256 = 0.00
1  01000000(bin) =  64(dec)   64/256 = 0.25
2  10000000(bin) = 128(dec)  128/256 = 0.50
3  11000000(bin) = 192(dec)  192/256 = 0.75


Recall that the any measured phase corresponds to $\theta = k / r$ where $k$ is sampled uniformly at random from $\{0, 1, \dots, r-1 \}$. Therefore, we can use the continued fractions algorithm to attempt to find $k$ and the order $r$. Python has this functionality built in. We can use the `fractions` module to turn a float into a `Fraction` object, for example:

In [22]:
Fraction(0.666)

Fraction(5998794703657501, 9007199254740992)

![פלט של תא הקוד הקודם](../docs/images/tutorials/shors-algorithm/extracted-outputs/95925dd5-7ba9-4746-b96e-ba50400fa5ac-1.avif)

## שלב 3: ביצוע באמצעות הפרימיטיבים של Qiskit
ראשית, אנו דנים במה שהיינו מקבלים תיאורטית אם היינו מריצים את המעגל הזה על סימולטור אידיאלי. להלן, יש לנו סט של תוצאות סימולציה של המעגל לעיל תוך שימוש ב-1024 shots. כפי שאנו יכולים לראות, אנו מקבלים התפלגות אחידה בקירוב על פני ארבעה מחרוזות ביט על פני קיוביטי הבקרה.

In [23]:
# Get fraction that most closely resembles 0.666
# with denominator < 15
Fraction(0.666).limit_denominator(15)

Fraction(2, 3)

This is much nicer. The order (r) must be less than N, so we will set the maximum denominator to be `15`:

In [24]:
# Rows to be displayed in a table
rows = []

for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append(
        [phase, f"{frac.numerator}/{frac.denominator}", frac.denominator]
    )

# Print the rows in a table
headers = ["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)

   Phase Fraction  Guess for r
0   0.00      0/1            1
1   0.25      1/4            4
2   0.50      1/2            2
3   0.75      3/4            4


![פלט של תא הקוד הקודם](../docs/images/tutorials/shors-algorithm/extracted-outputs/0d6d2702-02e4-47de-8f7e-0b256657ef0f-0.avif)

על ידי מדידת קיוביטי הבקרה, אנו מקבלים הערכת פאזה של שמונה ביט של האופרטור $M_a$. אנו יכולים להמיר את הייצוג הבינארי הזה לעשרוני כדי למצוא את הפאזה שנמדדה. כפי שאנו יכולים לראות מההיסטוגרמה לעיל, ארבעה מחרוזות ביט שונות נמדדו, וכל אחת מהן מתאימה לערך פאזה כדלקמן.

In [ ]:
# Sampler primitive to obtain the probability distribution
sampler = Sampler(backend)

# Turn on dynamical decoupling with sequence XpXm
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"
# Enable gate twirling
sampler.options.twirling.enable_gates = True

pub = transpiled_circuit
job = sampler.run([pub], shots=1024)

In [25]:
result = job.result()[0]
counts = result.data["out"].get_counts()

In [26]:
plot_histogram(counts, figsize=(35, 5))

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/559d7030-1f67-44e8-afa7-6afc7a334677-0.avif" alt="Output of the previous code cell" />

As we can see, we obtained the same bitstrings with highest counts. Since quantum hardware has noise, there is some leakage to other bitstrings, which we can filter out statistically.

In [27]:
# Dictionary of bitstrings and their counts to keep
counts_keep = {}
# Threshold to filter
threshold = np.max(list(counts.values())) / 2

for key, value in counts.items():
    if value > threshold:
        counts_keep[key] = value

print(counts_keep)

{'00000000': 58, '01000000': 41, '11000000': 42, '10000000': 40}


מכיוון שזה נותן שברים שמחזירים את התוצאה במדויק (במקרה זה, `0.6660000...`), זה יכול לתת תוצאות מסובכות כמו זו למעלה. אנו יכולים להשתמש במתודה `.limit_denominator()` כדי לקבל את השבר הדומה ביותר למספר העשרוני שלנו, עם מכנה מתחת לערך מסוים:

In [28]:
a = 2
N = 15

FACTOR_FOUND = False
num_attempt = 0

while not FACTOR_FOUND:
    print(f"\nATTEMPT {num_attempt}:")
    # Here, we get the bitstring by iterating over outcomes
    # of a previous hardware run with multiple shots.
    # Instead, we can also perform a single-shot measurement
    # here in the loop.
    bitstring = list(counts_keep.keys())[num_attempt]
    num_attempt += 1
    # Find the phase from measurement
    decimal = int(bitstring, 2)
    phase = decimal / (2**num_control)  # phase = k / r
    print(f"Phase: theta = {phase}")

    # Guess the order from phase
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator  # order = r
    print(f"Order of {a} modulo {N} estimated as: r = {r}")

    if phase != 0:
        # Guesses for factors are gcd(a^{r / 2} ± 1, 15)
        if r % 2 == 0:
            x = pow(a, r // 2, N) - 1
            d = gcd(x, N)
            if d > 1:
                FACTOR_FOUND = True
                print(f"*** Non-trivial factor found: {x} ***")


ATTEMPT 0:
Phase: theta = 0.0
Order of 2 modulo 15 estimated as: r = 1

ATTEMPT 1:
Phase: theta = 0.25
Order of 2 modulo 15 estimated as: r = 4
*** Non-trivial factor found: 3 ***


## Discussion

### Related work
In this section, we discuss other milestone work that has demonstrated Shor's algorithm on real hardware.

The seminal work [[3]](#references) from IBM&reg; demonstrated Shor's algorithm for the first time, factoring the number 15 into its prime factors 3 and 5 using a seven-qubit nuclear magnetic resonance (NMR) quantum computer. Another experiment [[4]](#references) factored 15 using photonic qubits. By employing a single qubit recycled multiple times and encoding the work register in higher-dimensional states, the researchers reduced the required number of qubits to one-third of that in the standard protocol, utilizing a two-photon compiled algorithm. A significant paper in the demonstration of Shor's algorithm is [[5]](#references), which uses Kitaev's iterative phase estimation [[8]](#references) technique to reduce the qubit requirement of the algorithm. Authors used seven control qubits and four cache qubits, together with the implementation of modular multipliers. This implementation, however, requires mid-circuit measurements with feed-forward operations and qubit recycling with reset operations. This demonstration was done on an ion-trap quantum computer.

More recent work [[6]](#references) focused on factoring 15, 21, and 35 on IBM Quantum&reg; hardware. Similar to previous work, researchers used a compiled version of the algorithm that employed a semi-classical quantum Fourier transform as proposed by Kitaev to minimize the number of physical qubits and gates. A most recent work [[7]](#references) also performed a proof-of-concept demonstration for factoring the integer 21. This demonstration also involved the use of a compiled version of the quantum phase estimation routine, and built upon the previous demonstration by [[4]](#references). Authors went beyond this work by using a configuration of approximate Toffoli gates with residual phase shifts. The algorithm was implemented on IBM quantum processors using only five qubits, and the presence of entanglement between the control and register qubits was verified successfully.

### Scaling of the algorithm

We note that RSA encryption typically involves key sizes on the order of 2048 to 4096 bits. Attempting to factor a 2048-bit number with Shor's algorithm will result in a quantum circuit with millions of qubits, including the error correction overhead and a circuit depth on the order of a billion, which is beyond the limits of current quantum hardware to execute. Therefore, Shor's algorithm will require either optimized circuit construction methods or robust quantum error correction to be practically viable for breaking modern cryptographic systems. We refer you to [[9]](#references) for a more detailed discussion on resource estimation for Shor's algorithm.

## Challenge

Congratulations for finishing the tutorial! Now is a great time to test your understanding. Could you try to construct the circuit for factoring 21? You can select an $a$ of your own choice. You will need to decide on the bit accuracy of the algorithm to choose the number of qubits, and you will need to design the modular exponentiation operators $M_a$. We encourage you to try this out yourself, and then read about the methodologies shown in Fig. 9 of [[6]](#references) and Fig. 2 of [[7]](#references).

In [ ]:
def M_a_mod21():
    """
    M_a (mod 21)
    """

    # Your code here
    pass

זה הרבה יותר נחמד. הסדר (r) חייב להיות קטן מ-N, אז נקבע את המכנה המקסימלי להיות `15`: